In [74]:
import h2o
import time
import seaborn
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator

%matplotlib inline

In [75]:
import h2o

In [76]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [77]:
from bayes_opt import BayesianOptimization

In [78]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,2 hours 12 mins
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.6
H2O cluster version age:,1 month and 2 days
H2O cluster name:,H2O_from_python_ssafini_xj04p5
H2O cluster total nodes:,1
H2O cluster free memory:,0
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [79]:
h2o.remove_all()

In [80]:
data = h2o.upload_file(r"C:\Users\ssafini\Desktop\HEC_A19\TIA\projet\winequality-red.csv",sep = ";", destination_frame="wine_df")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [81]:
data.describe()

Rows:1599
Cols:12




,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
type,real,real,real,real,real,real,real,real,real,real,real,int
mins,4.6,0.12,0.0,0.9,0.012,1.0,6.0,0.99007,2.74,0.33,8.4,3.0
mean,8.319637273295799,0.5278205128205145,0.27097560975609747,2.538805503439649,0.08746654158849286,15.874921826141357,46.46779237023142,0.9967466791744843,3.3111131957473443,0.6581488430268921,10.422983114446522,5.636022514071286
maxs,15.9,1.58,1.0,15.5,0.611,72.0,289.0,1.00369,4.01,2.0,14.9,8.0
sigma,1.7410963181276946,0.17905970415353528,0.19480113740531824,1.40992805950728,0.047065302010090085,10.46015696980971,32.89532447829907,0.0018873339538427274,0.15438646490354271,0.1695069795901101,1.0656675818473935,0.8075694397347051
zeros,0,0,132,0,0,0,0,0,0,0,0,0
missing,0,0,0,0,0,0,0,0,0,0,0,0
0,7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5.0
1,7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.2,0.68,9.8,5.0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.997,3.26,0.65,9.8,5.0


In [82]:
predictors  = [x for x in data.col_names if x not in ['quality']]

In [83]:
response  = "quality"

In [84]:
train[response] = train[response].asfactor()
valid[response] = valid[response].asfactor()
test[response] = test[response].asfactor()
print("Number of rows in train, valid and test set : ", train.shape[0], valid.shape[0], test.shape[0])

Number of rows in train, valid and test set :  967 321 311


In [85]:
#train, test = data.split_frame(ratios=[0.7])
train, valid, test = data.split_frame(ratios=[0.6,0.2], seed=1234)

In [86]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [87]:
def train_model(max_depth, 
                ntrees,
                min_rows, 
                learn_rate, 
                sample_rate, 
                col_sample_rate):
    params = {
        'max_depth': int(max_depth),
        'ntrees': int(ntrees),
        'min_rows': int(min_rows),
        'learn_rate':learn_rate,
        'sample_rate':sample_rate,
        'col_sample_rate':col_sample_rate
    }
    model = H2OGradientBoostingEstimator(nfolds=5,**params)
    model.train(x=predictors, y=response, training_frame=train)
    return -model.rmse()

In [88]:
bounds = {
    'max_depth':(5,10),
    'ntrees': (100,500),
    'min_rows':(10,30),
    'learn_rate':(0.001, 0.01),
    'sample_rate':(0.5,0.8),
    'col_sample_rate':(0.5,0.8)
}

In [89]:
optimizer = BayesianOptimization(
    f=train_model,
    pbounds=bounds,
    random_state=1,
)
optimizer.maximize(init_points=10, n_iter=50)

|   iter    |  target   | col_sa... | learn_... | max_depth | min_rows  |  ntrees   | sample... |
-------------------------------------------------------------------------------------------------
gbm Model Build progress: |███████████████████████████████████████████████| 100%
|  1        | -0.5907   |  0.6251   |  0.007483 |  5.001    |  16.05    |  158.7    |  0.5277   |
gbm Model Build progress: |███████████████████████████████████████████████| 100%
|  2        | -0.5849   |  0.5559   |  0.00411  |  6.984    |  20.78    |  267.7    |  0.7056   |
gbm Model Build progress: |███████████████████████████████████████████████| 100%
|  3        | -0.5377   |  0.5613   |  0.008903 |  5.137    |  23.41    |  266.9    |  0.6676   |
gbm Model Build progress: |███████████████████████████████████████████████| 100%
|  4        | -0.657    |  0.5421   |  0.002783 |  9.004    |  29.37    |  225.4    |  0.7077   |
gbm Model Build progress: |███████████████████████████████████████████████| 100%
|  5   

|  45       | -0.6661   |  0.5      |  0.001    |  10.0     |  10.0     |  443.0    |  0.5      |
gbm Model Build progress: |███████████████████████████████████████████████| 100%
|  46       | -0.706    |  0.8      |  0.001    |  10.0     |  30.0     |  312.6    |  0.8      |
gbm Model Build progress: |███████████████████████████████████████████████| 100%
|  47       | -0.4975   |  0.8      |  0.01     |  5.0      |  30.0     |  449.7    |  0.8      |
gbm Model Build progress: |███████████████████████████████████████████████| 100%
|  48       | -0.498    |  0.8      |  0.01     |  5.0      |  10.0     |  284.3    |  0.5      |
gbm Model Build progress: |███████████████████████████████████████████████| 100%
|  49       | -0.5733   |  0.5      |  0.01     |  10.0     |  30.0     |  202.1    |  0.5      |
gbm Model Build progress: |███████████████████████████████████████████████| 100%
|  50       | -0.7527   |  0.8      |  0.001    |  10.0     |  30.0     |  139.6    |  0.5      |
gbm Mod

In [90]:
optimizer.max

{'target': -0.2990458901088457,
 'params': {'col_sample_rate': 0.8,
  'learn_rate': 0.01,
  'max_depth': 10.0,
  'min_rows': 10.0,
  'ntrees': 470.09324639220443,
  'sample_rate': 0.8}}

In [113]:
gbm_tune = H2OGradientBoostingEstimator(
  col_sample_rate= 0.8,
  learn_rate= 0.01,
  max_depth= 10,
  min_rows= 10.0,
  ntrees= 470,
  sample_rate= 0.8
)

In [114]:
gbm_tune.train(x=predictors, y=response, training_frame=train, validation_frame=valid)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [115]:
gbm_tune.model_performance(train).rmse()

0.2979915585688224

In [116]:
gbm_tune.model_performance(valid).rmse()

0.5961260368482346

In [117]:
gbm_tune.model_performance(test).rmse()

0.6385850946142128

In [118]:
gbm = H2OGradientBoostingEstimator()
gbm.train(x=predictors, y=response, training_frame=train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [119]:
print(gbm)

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1572909938905_141760


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,50.0,14023.0,5.0,5.0,5.0,7.0,29.0,17.66




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 0.16750653396043771
RMSE: 0.40927562101893844
MAE: 0.31585724523913156
RMSLE: 0.06255560201191934
Mean Residual Deviance: 0.16750653396043771

Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2019-11-04 21:27:39,0.000 sec,0.0,0.789145,0.672306,0.622749
1,,2019-11-04 21:27:39,0.000 sec,1.0,0.752297,0.639229,0.565951
2,,2019-11-04 21:27:39,0.000 sec,2.0,0.721151,0.609928,0.520059
3,,2019-11-04 21:27:39,0.016 sec,3.0,0.692025,0.581823,0.478898
4,,2019-11-04 21:27:39,0.016 sec,4.0,0.669507,0.558629,0.448240
5,,2019-11-04 21:27:39,0.016 sec,5.0,0.647217,0.537402,0.418890
6,,2019-11-04 21:27:39,0.016 sec,6.0,0.629491,0.519943,0.396259
7,,2019-11-04 21:27:39,0.032 sec,7.0,0.612390,0.505381,0.375021
8,,2019-11-04 21:27:39,0.032 sec,8.0,0.596205,0.490064,0.355461
9,,2019-11-04 21:27:39,0.032 sec,9.0,0.583993,0.478547,0.341048



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,alcohol,843.688904,1.000000,0.363971
1,sulphates,330.955414,0.392272,0.142775
2,volatile acidity,281.781494,0.333987,0.121562
3,total sulfur dioxide,205.498779,0.243572,0.088653
4,free sulfur dioxide,118.924393,0.140958,0.051304
5,fixed acidity,112.835953,0.133741,0.048678
6,chlorides,110.459106,0.130924,0.047652
7,pH,97.845680,0.115974,0.042211
8,residual sugar,81.000473,0.096008,0.034944
9,citric acid,73.823807,0.087501,0.031848


In [127]:
gbm.model_performance(train).rmse()

0.4092755950930113

In [126]:
gbm.model_performance(valid).rmse()

0.6283658790637019

In [128]:
gbm.model_performance(test).rmse()

0.6549555232514734

# Grid Search:

In [108]:
from h2o.grid.grid_search import H2OGridSearch

In [109]:
gbm_grid = H2OGradientBoostingEstimator(
    ntrees = 3000,
    learn_rate = 0.01,
    stopping_rounds = 20,
    col_sample_rate = 0.7,
    sample_rate = 0.7,
    seed = 1234
) 

In [110]:
hyper_params = {'max_depth':[4,6,8,10,12]}
grid = H2OGridSearch(gbm_grid, hyper_params,
                         grid_id='depth_grid',
                         search_criteria={'strategy': "Cartesian"})

#Train grid search

In [111]:
grid.train(x=predictors, 
           y=response,
           training_frame=train,
           validation_frame=valid)

gbm Grid Build progress: |████████████████████████████████████████████████| 100%


In [112]:
print(grid)

    max_depth           model_ids    residual_deviance
0           8  depth_grid_model_3   0.3679210115158657
1          10  depth_grid_model_4  0.37529845115582056
2          12  depth_grid_model_5   0.3757855733074295
3           6  depth_grid_model_2   0.3809928726837449
4           4  depth_grid_model_1  0.38623498475780405



In [125]:
sorted_grid = grid.get_grid(sort_by='rmse')
print(sorted_grid)

    max_depth           model_ids                rmse
0           8  depth_grid_model_3  0.6065649277001315
1          10  depth_grid_model_4    0.61261607157813
2          12  depth_grid_model_5  0.6130135180462414
3           6  depth_grid_model_2  0.6172462010281998
4           4  depth_grid_model_1  0.6214780645829779



In [131]:
cv_gbm = H2OGradientBoostingEstimator(
    col_sample_rate= 0.8,
  learn_rate= 0.01,
  max_depth= 10,
  min_rows= 10.0,
  ntrees= 470,
  sample_rate= 0.8)
cv_gbm.train(x = predictors, y = response, training_frame = train, validation_frame=valid)



gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [133]:
gbm.model_performance(train).rmse()

0.4092755950930113

In [132]:
cv_gbm.model_performance(valid).rmse()

0.5929563403172124

In [134]:
gbm.model_performance(test).rmse()

0.6549555232514734

In [150]:
from h2o.automl import H2OAutoML

aml = H2OAutoML(max_models = 10, max_runtime_secs=100, seed = 1)
aml.train(x=predictors, y=response, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [151]:
lb = aml.leaderboard
lb

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_AutoML_20191104_215039,0.359968,0.599973,0.359968,0.440206,0.091911
StackedEnsemble_AllModels_AutoML_20191104_215039,0.361282,0.601067,0.361282,0.439031,0.0920819
XRT_1_AutoML_20191104_221115,0.362896,0.602408,0.362896,0.441078,0.0923315
XRT_1_AutoML_20191104_215039,0.362896,0.602408,0.362896,0.441078,0.0923315
XRT_1_AutoML_20191104_220851,0.362896,0.602408,0.362896,0.441078,0.0923315
DRF_1_AutoML_20191104_221115,0.363681,0.603059,0.363681,0.442818,0.0924653
DRF_1_AutoML_20191104_220851,0.363681,0.603059,0.363681,0.442818,0.0924653
DRF_1_AutoML_20191104_215039,0.363681,0.603059,0.363681,0.442818,0.0924653
GBM_4_AutoML_20191104_220851,0.379685,0.616186,0.379685,0.46198,0.0945107
GBM_4_AutoML_20191104_221115,0.379685,0.616186,0.379685,0.46198,0.0945107


In [152]:
from h2o.automl import H2OAutoML

aml = H2OAutoML(max_models = 10, max_runtime_secs=100, seed = 1)
aml.train(x=predictors, y=response, training_frame=train,leaderboard_frame = valid )

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [153]:
lb = aml.leaderboard
lb

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20191104_215039,0.343839,0.586378,0.343839,0.428857,0.0922593
StackedEnsemble_BestOfFamily_AutoML_20191104_215039,0.344798,0.587195,0.344798,0.431124,0.0923755
DRF_1_AutoML_20191104_221115,0.347866,0.589801,0.347866,0.426467,0.0930789
DRF_1_AutoML_20191104_221244,0.347866,0.589801,0.347866,0.426467,0.0930789
DRF_1_AutoML_20191104_220851,0.347866,0.589801,0.347866,0.426467,0.0930789
DRF_1_AutoML_20191104_215039,0.347866,0.589801,0.347866,0.426467,0.0930789
GBM_4_AutoML_20191104_220851,0.361211,0.601008,0.361211,0.453241,0.0939886
GBM_4_AutoML_20191104_221244,0.361211,0.601008,0.361211,0.453241,0.0939886
GBM_4_AutoML_20191104_221115,0.361211,0.601008,0.361211,0.453241,0.0939886
GBM_4_AutoML_20191104_215039,0.361211,0.601008,0.361211,0.453241,0.0939886


In [155]:
from h2o.automl import H2OAutoML

aml = H2OAutoML(max_models = 10, max_runtime_secs=100, seed = 1)
aml.train(x=predictors, y=response, training_frame=train,validation_frame=valid,leaderboard_frame = test )

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [156]:
lb = aml.leaderboard
lb

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
DRF_1_AutoML_20191104_221115,0.388294,0.623132,0.388294,0.466307,0.0988544
DRF_1_AutoML_20191104_221401,0.388294,0.623132,0.388294,0.466307,0.0988544
DRF_1_AutoML_20191104_221244,0.388294,0.623132,0.388294,0.466307,0.0988544
DRF_1_AutoML_20191104_220851,0.388294,0.623132,0.388294,0.466307,0.0988544
DRF_1_AutoML_20191104_215039,0.388294,0.623132,0.388294,0.466307,0.0988544
StackedEnsemble_AllModels_AutoML_20191104_215039,0.390807,0.625146,0.390807,0.471032,0.0989781
StackedEnsemble_BestOfFamily_AutoML_20191104_215039,0.391708,0.625866,0.391708,0.47271,0.0990705
GBM_3_AutoML_20191104_221401,0.400076,0.632516,0.400076,0.495361,0.100226
GBM_3_AutoML_20191104_215039,0.400076,0.632516,0.400076,0.495361,0.100226
GBM_3_AutoML_20191104_220851,0.400076,0.632516,0.400076,0.495361,0.100226
